# Quantum 2x2 Games

This notebook has been developed in order to formulate and analyze the quantum version of two canonical Game Theory problems: Prisoners' Dilemma and Battle of Sexes.

In [42]:
import numpy as np
import cmath
from scipy import linalg
import matplotlib.pyplot as plt
import pandas as pd

The action space of a quantum 2x2 game is SU(2), i.e. the Hilbert space of unitary, trace-preserving 2x2 matrices.

A generic matrix in SU(2) can be written as a function of 2 parameters: <br>

$$
\hat{U}(\theta,\phi) = \begin{pmatrix}
e^{i\phi}\cos\theta/2 & \sin\theta/2 \\ -\sin\theta/2 & e^{-i\phi}\cos\theta/2
\end{pmatrix} \quad \theta\in[0,\pi], \phi\in[0,\frac{\pi}{2}] 
$$

and so it is possible to write a function that construct a generic quantum strategy accessible by one of the players.

In [2]:
def quantum_strategy(theta, phi):
    
    qs = np.zeros([2,2], dtype=complex)
    qs[0,0] = cmath.exp(1j*phi)*np.cos(theta/2)
    qs[0,1] = np.sin(theta/2)
    qs[1,0] = -np.sin(theta/2)
    qs[1,1] = cmath.exp(-1j*phi)*np.cos(theta/2)
    
    return qs

Also the original pure strategies C and D can be written in this way:

$$
\hat{C} = \hat{U}(0,0) = \begin{pmatrix}1&0\\0&1\end{pmatrix} \qquad
\hat{D} = \hat{U}(\pi,0) = \begin{pmatrix}0&1\\-1&0\end{pmatrix}
$$

In [3]:
C = quantum_strategy(0, 0)
D = quantum_strategy(np.pi, 0)

Usually, in these kind of games, each player has access to two pure strategies, C for cooperating and D for defecting, and to the corresponding payoff bymatrix:

$$
\begin{pmatrix}
P_{CC,A},P_{CC,B} & P_{CD,A},P_{CD,B} \\ P_{DC,A},P_{DC,B} & P_{DD,A},P_{DD,B} \\
\end{pmatrix}
$$

The states of the (quantum) system are defined, as said, in SU(2), and are expressed as column vectors with four elements, written in the basis given by {|CC>, |CD>, |DC>, |DD>}, i.e. the computational basis (for example, |CC> corresponds to the vector (1,0,0,0), and so on).

In order to construct an effective quantum game, we need to provide also some kind of entanglement between the various states, and this will be possible introducing an operator that depends on a "entanglement measure"

$$
\hat{J} = \exp\left(-i\gamma\hat{D}\otimes\hat{D}/2\right) \qquad \gamma\in[0,\frac{\pi}{2}]
$$

such that the final state of the system, after the application of the actions chosen by the players, but before the effective measure of the qubits, will be given by:

$$
|\psi_f> = \hat{J}^\dagger\left(\hat{U}_A\otimes\hat{U}_B\right)\hat{J}|CC>
$$

In [4]:
def compute_final_state( strategyA, strategyB, gamma ):
    
    # Construct matrix J
    J = linalg.expm(-1j*gamma*np.kron(C,C)/2)
    # J^+ is just J's adjoint
    J1 = J.conjugate().transpose()
    # UA x UB is the kronecker product between the two quantum strategies
    UAB = np.kron(strategyA, strategyB)
    
    # Apply the operators one by one
    psi_f = np.matmul(J,np.array([1,0,0,0]))
    psi_f = np.matmul(UAB, psi_f)
    psi_f = np.matmul(J1, psi_f)
    
    return psi_f

In the end, since the only way to find the best set of strategies to follow is to check the payoff, the formula to compute the expected utility for player i is:

$$
<i> = P_{CC,i}\cdot|<CC|\psi_f>|^2 + P_{CD,i}\cdot|<CD|\psi_f>|^2 + P_{DC,i}\cdot|<DC|\psi_f>|^2 + P_{DD,i}\cdot|<DD|\psi_f>|^2
$$

In [5]:
def expected_payoffs( state, payoff_bymatrix ):
    
    payoffA, payoffB = 0, 0
    
    for i in range(2):
        for j in range(2):
            payoffA += payoff_bymatrix['A'][i,j]*(abs(state[2*i+j])**2)
            payoffB += payoff_bymatrix['B'][i,j]*(abs(state[2*i+j])**2)
    
    return [payoffA, payoffB]

Just to check the correctness of our code, we can rely on the fact that selecting $\gamma=0$ we should obtain the original classical game! <br>
So, if for example, we ask the two players to choose the joint strategy {DC}, then as final state we should get |DC> in the computational basis, that is (0,0,1,0).

In [6]:
compute_final_state( strategyA=D, strategyB=C, gamma=0 ).astype(int)

/home/mattia/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: ComplexWarning: Casting complex values to real discards the imaginary part
  """Entry point for launching an IPython kernel.


array([ 0,  0, -1,  0])

Note: the -  sign in the coefficient is irrelevant because what matters is its square module, i.e. $|-1|^2$, that represents the probability of finding the system in the state CD.

So everything seems working fine!

# Quantum Prisoners' Dilemma

The first problem that we are going to simulate as a quantum system is the traditional Prisoners' Dilemma.

In [7]:
# Specify the payoff matrix
payoff_matrix_pd = {'A' : np.array([[3,0],[5,1]]), 'B' : np.array([[3,5],[0,1]]) }

In [8]:
fs = compute_final_state( strategyA=D, strategyB=C, gamma=0 )

In [9]:
print(fs)
expected_payoffs( fs, payoff_matrix_pd )

[ 6.123234e-17+0.j  0.000000e+00+0.j -1.000000e+00+0.j  0.000000e+00+0.j]


[5.0, 1.1248198369963932e-32]

It may be interesting, now, to analyze the different payoffs of player A in particular configurations. Then the game, at least for the moment is symmetric, and so we are allowed to draw the same conclusions also for player B.

Let's set $\gamma=0$ and construct a discrete 4-dimensional grid for all the possible values of the parameters $\theta$ and $\phi$ for both players.

In [57]:
thetaA = np.arange(0, np.pi, 0.1)
phiA   = np.arange(0, np.pi/2, 0.1)
thetaB = np.arange(0, np.pi, 0.1)
phiB   = np.arange(0, np.pi/2, 0.1)

grid = np.zeros(shape=(32*16*32*16,5))
s = 0

for i in thetaA:
    for j in phiA:
        for k in thetaB:
            for l in phiB:
                print(32*16*32*16-s)
                sA = quantum_strategy(thetaA, phiA[0])
                sB = quantum_strategy(thetaB[0], phiB[0])
                fs = compute_final_state(sA, sB, 0)
                utility = expected_payoffs(fs, payoff_matrix_pd)
                grid[s] = [thetaA[0], phiA[0], thetaB[0], phiB[0], utility[0]]
                s +=1
                

262144


TypeError: only length-1 arrays can be converted to Python scalars

In [56]:
pd.DataFrame(grid, columns=["tA","pA","tB","pB","u",])

,tA,pA,tB,pB,u
0,0.0,0.0,0.0,0.0,3.0
1,0.0,0.0,0.0,0.0,3.0
2,0.0,0.0,0.0,0.0,3.0
3,0.0,0.0,0.0,0.0,3.0
4,0.0,0.0,0.0,0.0,3.0
...,...,...,...,...,...
262139,0.0,0.0,0.0,0.0,3.0
262140,0.0,0.0,0.0,0.0,3.0
262141,0.0,0.0,0.0,0.0,3.0
262142,0.0,0.0,0.0,0.0,3.0
